In [1]:
import sys
sys.path.append("../")
import os
import vqe_experiment as ve
import numpy as np

In [ ]:
# define Hamiltonian: list of coefficients, list of Paulis, initial bitstring

# here: molecular Hamiltonian, parity mapping, two-qubit reduction
atom_string = "H 0 0 0; H 0 0 2.0"              # PySCF atom string
num_orbitals = 1                                 # number of spatial orbitals
coeffs, paulis, HF_bitstring = ve.molecule(atom_string, new_num_orbitals=num_orbitals)

In [ ]:
coeffs, paulis, HF_bitstring

In [ ]:
# number of qubits
N = len(paulis[0])
N

In [5]:
# configure underlying VQE structure, passed as keyword argument dictionary

# a function which returns a parameterized Qiskit circuit (ansatz) and the number of parameters
ansatz_func = ve.efficientsu2_full
ansatz_reps = 1
# if True, the initial HF_bitstring will be appended after ansatz (layer of X gates where there are 1s)
# if False, HF_bitstring will be initialized before ansatz
init_last = False

vqe_kwargs = {
    "ansatz_func": ansatz_func,
    "ansatz_reps": ansatz_reps,
    "init_last": init_last,
    "HF_bitstring": HF_bitstring
}

In [ ]:
ansatz, num_params = ansatz_func(N, ansatz_reps)
print(num_params)

In [ ]:
ansatz.draw(fold=-1)

CAFQA parameters are represented by a list $[x_0, x_1, x_2, ...]$ where $x_i$ parametrizes the $i$-th gate in the ansatz and can only take the discrete values $\{0,1,2,3\}$. Typically the parameterized gates are rotation gates, $R_y$ or $R_z$, so that $x_i$ corresponds to a rotation angle $x_i \cdot \pi/2$.

In [ ]:
# configure algorithm

budget = 100                                                 # number of iterations
guess = list(map(int, np.random.randint(0, 4, num_params)))  # initial param guess
guess

In [9]:
# get VQE circuit guess parameters (* pi/2)
qc = ve.vqe_circuit(
    N, 
    np.array(guess)*np.pi/2, 
    paulis, 
    ansatz_func=ansatz_func, 
    ansatz_reps=ansatz_reps, 
    init_last=init_last,
    HF_bitstring=HF_bitstring
    )

In [ ]:
qc.draw(fold=-1)

In [11]:
# configure output

save_dir = "cafqa_example_out/"
os.system(f"mkdir -p {save_dir}")
# the output files are relative to save_dir
result_file = "result.txt"
loss_file = "cafqa_loss.txt"
params_file = "cafqa_params.txt"

In [ ]:
# run CAFQA

# hypermapper changes the standard output to write to log files, so it needs to be
# changed back after CAFQA (otherwise printing breaks)

stdout = sys.stdout
cafqa_energy, cafqa_params = ve.run_cafqa(
    n_qubits=N,
    coeffs=coeffs,
    paulis=paulis,
    param_guess=list(guess),
    budget=budget,
    save_dir=save_dir,
    loss_file=loss_file,
    params_file=params_file,
    vqe_kwargs=vqe_kwargs
)
sys.stdout = stdout